In [0]:
%pip install -U -qqq \
databricks-sdk==0.65.0 \
databricks-langchain==0.7.1 \
databricks-vectorsearch==0.57 \
mlflow==3.3.2 \
langgraph==0.3.4 \
langchain==0.3.27 \
langchain-core==0.3.75 \
langchain-community==0.3.29 \
langchain-huggingface==0.3.1 \
langchain-tavily==0.2.11 \
databricks-agents \
python-docx==1.2.0 openpyxl==3.1.5 pillow==12.0.0 \
pypdf==6.0.0 requests==2.32.5 pandas==2.2.3 numpy==1.26.4 \
sentence-transformers==5.1.0 \
"transformers>=4.49.0,<4.51" "tokenizers>=0.21.2,<0.22" \
torch==2.4.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu \
google-cloud-vision==3.10.2 google-auth==2.40.3 google-genai\
rapidfuzz==3.14.1

In [0]:
dbutils.library.restartPython() 

In [0]:

import os
from databricks.sdk.core import DatabricksError
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.vectorsearch import EndpointStatusState, EndpointType
from databricks.sdk.service.serving import EndpointCoreConfigInput, EndpointStateReady
from databricks.sdk.errors import ResourceDoesNotExist, NotFound, PermissionDenied

CURRENT_FOLDER = os.getcwd()
w = WorkspaceClient()

# カタログとスキーマ
UC_CATALOG = 'hhhd_demo_itec'
UC_SCHEMA = 'commuting_allowance'

# モデル
UC_MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.commuting_allowance_model"

# カタログを作成
try:
    _ = w.catalogs.get(UC_CATALOG)
    print(f"PASS: UC catalog `{UC_CATALOG}` exists")
except NotFound as e:
    print(f"`{UC_CATALOG}` does not exist, trying to create...")
    try:
        _ = w.catalogs.create(name=UC_CATALOG)
    except PermissionDenied as e:
        print(f"FAIL: `{UC_CATALOG}` does not exist, and no permissions to create.  Please provide an existing UC Catalog.")
        raise ValueError(f"Unity Catalog `{UC_CATALOG}` does not exist.")
        
# スキーマを作成
try:
    _ = w.schemas.get(full_name=f"{UC_CATALOG}.{UC_SCHEMA}")
    print(f"PASS: UC schema `{UC_CATALOG}.{UC_SCHEMA}` exists")
except NotFound as e:
    print(f"`{UC_CATALOG}.{UC_SCHEMA}` does not exist, trying to create...")
    try:
        _ = w.schemas.create(name=UC_SCHEMA, catalog_name=UC_CATALOG)
        print(f"PASS: UC schema `{UC_CATALOG}.{UC_SCHEMA}` created")
    except PermissionDenied as e:
        print(f"FAIL: `{UC_CATALOG}.{UC_SCHEMA}` does not exist, and no permissions to create.  Please provide an existing UC Schema.")
        raise ValueError("Unity Catalog Schema `{UC_CATALOG}.{UC_SCHEMA}` does not exist.")

In [0]:
import mlflow
from mlflow.models.resources import (
  DatabricksServingEndpoint,
  DatabricksSQLWarehouse,
  DatabricksFunction,
  DatabricksGenieSpace,
  DatabricksTable,
  DatabricksUCConnection
)

input_example = { 
    "messages": [
        {
            "role": "user",
            "content": "AIエージェントとは？",
        }
    ]
}

AGENT_NAME1 = "CommuteWorkerAgent"
AGENT_NAME2 = "CommuteSuperVisorAgent"
with mlflow.start_run():
  logged_agent_info = mlflow.pyfunc.log_model(
    name=AGENT_NAME1,
    python_model=AGENT_NAME1,
    input_example=input_example,
    pip_requirements=[
      # === Core MLflow / LangGraph / LangChain ===
      "mlflow==3.3.2",
      "langgraph==0.3.4",
      "langgraph-supervisor==0.0.4",
      "databricks-langchain==0.7.1",
      "langchain==0.3.27",
      "langchain-core==0.3.75",
      "langchain-community==0.3.29",

      # === Databricks integration ===
      "databricks-sdk==0.65.0",
      "databricks-vectorsearch==0.57",

      # === Model / Embedding ===
      "sentence-transformers==5.1.0",
      "transformers>=4.49.0,<4.51",
      "tokenizers>=0.21.2,<0.22",
      "sentencepiece>=0.2.0,<0.3",

      # === IO / Utility ===
      "pypdf==6.0.0",
      "python-docx==1.2.0",
      "openpyxl==3.1.5",
      "pandas==2.2.3",
      "numpy==1.26.4",
      "requests==2.32.5",

      # === Google Cloud APIs ===
      "google-cloud-vision==3.10.2",
      "google-auth==2.40.3",

      # === LangChain拡張 ===
      "langchain-huggingface==0.3.1",
      "langchain-tavily==0.2.11",

      # === その他ユーティリティ ===
      "mutagen==1.47.0",
    ],
  )

In [0]:
import threading
from mlflow.genai.scorers import Safety

# テストデータ
eval_data = []
# data = {
#   "inputs": {"query": "通勤手当-新規(true) .xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "26072から25848までの通勤経路を朝8時45分到着で検索してください"},
# }
data = {
  "inputs": {"query": "658-0033 兵庫県神戸市東灘区向洋町西５丁目９ の最寄り駅を教えてください。"},
}
# data = {
#   "inputs": {"query": "阪神線の野田駅から、中津（大阪メトロ）までの6か月定期持っています。今年の10月1日購入しで、来月末に払い戻しする場合はいくらかえってくれますか？"},
# }
# data = {
#   "inputs": {"query": "住吉(兵庫県・東海道)から大阪市海老江駅まで尼崎(東海道本線)で乗り換えのルートを教えてください。"},
# }
# data = {
#   "inputs": {"query": "26413から29162までのルートを教えてください。"},
# }
# data = {
#   "inputs": {"query": "申請書（PiTaPa取り消し）.xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "申請書（他候補なし_自社線含む）.xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "申請書（他候補あり_自社線含む）.xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "申請書（新経路_JR）.xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "申請書（自社線との比較＆払戻あり）.xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "野田駅から中津駅までの通勤経路を検索してください"}
# }
# data = {
#   "inputs": {"query": "通勤手当-変更4.xlsx こちらの通勤手当変更申請を査定してください。"}
# }
# data = {
#   "inputs": {"query": "神戸市東灘区光洋町西5丁目8 の最寄り駅を教えてください。"}
# }
eval_data.append(data)

# モデルロード（1回だけ）
logged_model = mlflow.get_logged_model(logged_agent_info.model_id)
model = mlflow.pyfunc.load_model(f"models:/{logged_model.model_id}")

def _call_model_with_retry(messages_for_llm):
  try:
    return model.predict(messages_for_llm)
  except Exception as e:
    msg = str(e)
    print(f"Error: {msg}")
    return msg

def evaluate_model(query: str, **kwargs):
  # 余分なキーが来ても壊れないよう **kwargs を受ける
  messages_for_llm = {
    "messages": [
      {"role": "user", "content": query}
    ]
  }
  return _call_model_with_retry(messages_for_llm)

def _evaluate_batch_single_stage(batch):
  return mlflow.genai.evaluate(
    data=batch,
    predict_fn=evaluate_model,
    scorers=[Safety()]
  )

try:
    res = _evaluate_batch_single_stage(eval_data)
except Exception as e:
    msg = str(e)
    print(f"Error: {msg}")

In [0]:
from databricks import agents
import time
from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

mlflow.set_registry_uri("databricks-uc")
UC_MODEL_NAME = f"{UC_CATALOG}.multi_agents.commute-worker-agent-test"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

# Deploy to enable the Review APP and create an API endpoint
deployment_info = agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "docs"}
)

# Wait for the Review App to be ready
print("\nWaiting for endpoint to deploy.  This can take 15 - 20 minutes.", end="")
while w.serving_endpoints.get(deployment_info.endpoint_name).state.ready == EndpointStateReady.NOT_READY or w.serving_endpoints.get(deployment_info.endpoint_name).state.config_update == EndpointStateConfigUpdate.IN_PROGRESS:
    print(".", end="")
    time.sleep(30)

print(f"Endpoint {deployment_info.endpoint_name} is now ready!")